In [203]:
from IPython.core.display import display, HTML
from IPython.display import display, HTML
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
import re

C:\Users\sodji\AppData\Local\Temp\ipykernel_2332\111908625.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [205]:
def scrape_wikipedia_table(url):
    start_time = time.time()
    # Envoyer une requête HTTP pour obtenir le contenu de la page
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    print(response)

    # Trouver le premier tableau dans la page
    table = soup.find('table', {'class': 'wikitable'})

    # Extraire les données du tableau
    data = []
    headers = []
    for row in table.find_all('tr'):
        cells = row.find_all(['th', 'td'])
        cell_data = [cell.get_text(strip=True) for cell in cells]

        if not headers:
            headers = cell_data  # En-tête du tableau
        else:
            data.append(cell_data)

    # Convertir les données en DataFrame
    df = pd.DataFrame(data, columns=headers)

    # Normaliser les noms de colonnes pour la recherche
    normalized_headers = [col.lower().replace(' ', '') for col in headers]

    # Trouver les indices des colonnes "others" et "samplesize"
    try:
        others_index = normalized_headers.index('others')
    except ValueError:
        others_index = None

    try:
        sample_size_index = normalized_headers.index('samplesize')
    except ValueError:
        sample_size_index = None

    # Calculer L1 et L2
    L1 = others_index if others_index is not None else len(headers)
    L2 = sample_size_index + 1 if sample_size_index is not None else 0

    # Réorganiser les colonnes
    if L1 > L2:
        part1 = headers[:L2]
        part2 = headers[L2:L1]
        part2_renamed = [f"Part{i+1}" for i in range(len(part2))]

        # Réorganiser le DataFrame
        df = df[part1 + part2]
        df.columns = part1 + part2_renamed

    end_time = time.time()
    print(f"Web Scraping processing took: {end_time - start_time:.4f} seconds")

    return df


### Pour afficher les dataframes

In [7]:

def process_urls(urls):
    # Dictionnaire pour stocker les DataFrames
    data_frames = {}

    # Expression régulière pour extraire l'année et le pays de l'URL
    pattern = re.compile(r'(\d{4})_([A-Za-z_]+)')

    for url in urls:
        # Extraire l'année et le pays de l'URL
        match = pattern.search(url)
        if match:
            year = match.group(1)
            country = match.group(2).replace('_', ' ')

            # Utiliser la fonction scrape_wikipedia_table pour obtenir le DataFrame
            df = scrape_wikipedia_table(url)

            # Nommer la variable selon le pays et l'année
            variable_name = f"data_poll_{country}_{year}"

            # Stocker le DataFrame dans le dictionnaire
            data_frames[variable_name] = df

    return data_frames

### Date standardization 

In [9]:
def clean_date_column(df, file_name):
    # Extraire l'année à partir du nom du fichier
    match = re.search(r'_(\d{4})', file_name)
    year = match.group(1) if match else ""
    
    # Dictionnaire des mois en anglais
    months_dict = {
        "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6,
        "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12
    }
    
    # Identifier les colonnes contenant le mot "date" sous toutes ses formes
    date_columns = [col for col in df.columns if re.search(r'date', col, re.IGNORECASE)]
    
    # Nettoyer les colonnes de dates
    for col in date_columns:
        new_values = []
        for value in df[col].astype(str):
            # Chercher le premier mot après les chiffres (exemple: "15-Apr" ou "15 April")
            m = re.search(r'\d+\s*[-–]?\s*\d*\s*([A-Za-z]+)', value)
            if m:
                month_str = m.group(1)[:3]  # Prendre les 3 premières lettres pour uniformiser
                if month_str in months_dict:
                    # Format : "mois-année", par exemple "4-2015"
                    new_values.append(f"{months_dict[month_str]}/{year}")
                else:
                    new_values.append(value)
            else:
                new_values.append(value)
        df[col] = new_values
    
    return df


### Fonction pour netoyer les organisations

In [11]:
# Charger le fichier à nouveau
def clean_polling_column(df):
    # Compter le nombre total de colonnes
    total_columns = len(df.columns)
    
    # Prendre le nom de toutes les colonnes
    column_names = list(df.columns)
    
    # Identifier les colonnes contenant le mot "Poll" sous toutes ses formes
    poll_columns = [col for col in column_names if re.search(r'poll', col, re.IGNORECASE)]
    
    # Nettoyer les colonnes correspondantes
    for col in poll_columns:
        new_values = []
        for value in df[col].astype(str):
            # Extraire le texte avant "[]" ou "/"
            match = re.match(r'([^/\[]+)', value)
            cleaned_value = match.group(1).strip() if match else value.strip()
            new_values.append(cleaned_value)
        
        df[col] = new_values
    
    return df


### Fonction pour supprimer les '%'

In [13]:
def remove_percentage(df):
    # Identifier les colonnes contenant le mot "Part" sous toutes ses formes
    part_columns = [col for col in df.columns if re.search(r'part', col, re.IGNORECASE)]
    
    # Nettoyer les colonnes correspondantes
    for col in part_columns:
        df[col] = df[col].astype(str).str.replace('%', '', regex=True)
    
    return df



### Fonction pour remplacer les valeurs manquantes par 0

In [15]:
def replace_empty_and_underscore(df):
    """
    Fonction pour remplacer tous les "_" et les valeurs vides ("") par 0 dans un DataFrame.
    
    Arguments:
    df -- DataFrame contenant les données.
    
    Retourne:
    Un DataFrame nettoyé.
    """
    
    df.replace(["_", ""], 0, inplace=True)
    
    return df

In [16]:
def reorganize_dataframe_with_values(df):
    # Longueur totale des colonnes
    M = len(df.columns)
    
    # Identifier la colonne contenant "Sample" sous toutes ses formes
    sample_col = next((col for col in df.columns if re.search(r'sample', col, re.IGNORECASE)), None)
    
    if sample_col:
        # Trouver l'index de la colonne "Sample"
        l = df.columns.get_loc(sample_col) + 1  # Inclure la colonne elle-même
    else:
        return df  # Retourner le dataframe inchangé si "Sample" n'est pas trouvé

    # Identifier la colonne contenant "Poll" sous toutes ses formes
    poll_col = next((col for col in df.columns if re.search(r'poll', col, re.IGNORECASE)), None)
    
    if poll_col:
        # Trouver la ligne contenant "general election" dans la colonne "Poll"
        election_row = df[df[poll_col].str.contains("election", case=False, na=False)]
        
        if not election_row.empty:
            # Sélectionner la première occurrence
            row_index = election_row.index[0]

            # Prendre les valeurs des colonnes de l à M dans cette ligne
            values_list = df.iloc[row_index, l:M].tolist()
            m = len(values_list)

            # Ajouter m nouvelles colonnes et les remplir avec les valeurs respectives
            for i in range(m):
                df[f"Part{i+1}_result"] = values_list[i]
            

    return df


In [17]:
def nettoyer_dataframe_1(df):
    # Supprimer les lignes contenant "GB" dans n'importe quelle colonne
    df = df[~df.astype(str).apply(lambda x: x.str.contains("GB", na=False)).any(axis=1)]
    
    # Remplacer les valeurs "None" par 0
    df.replace("None", 0, inplace=True)
    # Remplacer toutes les valeurs manquantes (NaN) par 0
    df.fillna(0, inplace=True)
    
    return df

In [18]:
# Fonction pour nettoyer les colonnes inutiles
def nettoyer_dataframe_2(df):
    # Liste des noms de colonnes utiles
    col_list = ["poll_date", "sample_size", "polling_organization"]

    # Trouver l'index de "Part1" pour déterminer la longueur L
    if "Part1" in df.columns:
        L = df.columns.get_loc("Part1")
    else:
        L = len(df.columns)  # Si "Part1" n'existe pas, prendre toutes les colonnes

    # Renommer les colonnes en fonction des règles
    new_columns = df.columns.tolist()
    for i in range(L):
        col = new_columns[i]
        if "Date" in col:
            new_columns[i] = col_list[0]
        elif "Poll" in col:
            new_columns[i] = col_list[2]
        elif "Sample" in col:
            new_columns[i] = col_list[1]

    # Appliquer les nouveaux noms de colonnes
    df.columns = new_columns

    # Supprimer les colonnes qui ne sont pas dans col_list
    cols_to_keep = col_list + df.columns[L:].tolist()  # Garder les colonnes renommées et celles après L
    df = df[cols_to_keep]

    return df


In [19]:
def renommer_colonnes(df):
    # Trouver l'index de "Part1"
    if "Part1" in df.columns:
        l = df.columns.get_loc("Part1")
    else:
        return df  # Si "Part1" n'existe pas, retourner le dataframe inchangé

    # Déterminer la longueur totale des colonnes
    L = len(df.columns)

    # Renommer les colonnes à partir de l
    new_columns = df.columns.tolist()
    candidate_index = 1  # Index des candidats
    candidate = 1

    for i in range(l, L):
        col = new_columns[i]
        if col.startswith("Part") and "_result" not in col:
            new_columns[i] = f"prediction_result_candidate_{candidate}"
            candidate += 1
        elif col.startswith("Part") and "_result" in col:
            new_columns[i] = f"final_result_candidate_{candidate_index}"
            candidate_index += 1  # Augmenter l'index seulement après avoir traité un "PartX_result"

    # Appliquer les nouveaux noms de colonnes
    df.columns = new_columns
    return df



### Resume

In [21]:
def cleaner (df, file_name):
    data = clean_date_column(df, file_name)
    data = clean_polling_column(data)
    data = remove_percentage(data)
    data = replace_empty_and_underscore(data)
    data = reorganize_dataframe_with_values(data)
    data = nettoyer_dataframe_1(data)
    data = nettoyer_dataframe_2(data)
    data = renommer_colonnes(data)
    return data
    

### Ajoute des parties et identités des parties

In [23]:
def enrichir_dataframe(df, party_dict):
    # Identifier les colonnes contenant "prediction"
    prediction_cols = [col for col in df.columns if "prediction" in col]
    l = len(prediction_cols)  # Longueur des colonnes "prediction"

    # Ajouter les colonnes "identity_candidate_X"
    for i in range(1, l + 1):
        candidate_col = f"identity_candidate_{i}"
        df[candidate_col] = list(party_dict.keys())[i - 1] if i - 1 < len(party_dict) else "Unknown"

    # Ajouter les colonnes "political_learning_candidate_X"
    for i in range(1, l + 1):
        learning_col = f"political_learning_candidate_{i}"
        df[learning_col] = list(party_dict.values())[i - 1] if i - 1 < len(party_dict) else "Unknown"

    return df

### Ordonner les colonnes

In [25]:
def reordonner_colonnes(df):
    # Colonnes fixes qui ne changent pas de position
    fixed_columns = ['poll_date', 'sample_size', 'polling_organization']

    # Identifier les colonnes contenant "prediction"
    prediction_cols = [col for col in df.columns if "prediction" in col]
    l = len(prediction_cols)  # Nombre total de colonnes de prédiction

    # Création d'une nouvelle liste pour stocker l'ordre des colonnes
    reordered_columns = fixed_columns[:]

    # Réorganisation des colonnes des candidats
    for i in range(1, l + 1):
        candidate_cols = [col for col in df.columns if f"candidate_{i}" in col]
        reordered_columns.extend(candidate_cols)

    # Réordonner le dataframe selon la nouvelle structure des colonnes
    df = df[reordered_columns]

    return df

In [26]:
def extract_numbers_from_column(df):
    """
    Extrait uniquement les chiffres de la colonne spécifiée d'un DataFrame.
    
    :param df: DataFrame contenant la colonne à traiter
    :param column_name: Nom de la colonne où extraire les chiffres
    :return: DataFrame avec la colonne transformée
    """
     # Identifier les colonnes contenant "prediction"
    prediction_cols = [col for col in df.columns if "prediction" in col]
    l = len(prediction_cols)  # Longueur des colonnes "prediction"

    df_copy = df.copy()

    for i in range(0, l):
        column_name = prediction_cols [i]
        df_copy[column_name] = df_copy[column_name].astype(str).apply(lambda x: ''.join(re.findall(r'\d+', x)))
    return df_copy

In [27]:
def process_column_type_conversion(df):
    """
    Fonction qui prend en argument un DataFrame et :
      - Convertit la colonne 'sample_size' en numerique.
      - Parcourt toutes les colonnes pour rechercher celles contenant le mot
        'prediction' ou 'final' (peu importe la casse ou si le mot est collé à d'autres caractères)
        et les convertit en float.
      - Retourne le DataFrame modifié.
    """

    columns = df.columns
    if 'sample_size' in columns:
        df['sample_size'] = pd.to_numeric(df['sample_size'].astype(str).str.replace(',', '', regex=False), errors='coerce')
    # Parcours de toutes les colonnes du DataFrame
    for col in df.columns:
        # Vérifie si le nom de la colonne contient 'prediction' ou 'final', sans tenir compte de la casse
        if 'prediction' in col.lower() or 'final' in col.lower():
            df[col] = pd.to_numeric(df[col].astype(str), errors='coerce')    
    return df


### Application

### URL

In [207]:
urls = [
    "https://en.wikipedia.org/wiki/Opinion_polling_for_the_2024_United_Kingdom_general_election",
    "https://en.wikipedia.org/wiki/Opinion_polling_for_the_2019_United_Kingdom_general_election",
    "https://en.wikipedia.org/wiki/Opinion_polling_for_the_2017_United_Kingdom_general_election",
    "https://en.wikipedia.org/wiki/Opinion_polling_for_the_2015_United_Kingdom_general_election",
    "https://en.wikipedia.org/wiki/Opinion_polling_for_the_2010_United_Kingdom_general_election",
    "https://en.wikipedia.org/wiki/Opinion_polling_for_the_2005_United_Kingdom_general_election",
    "https://en.wikipedia.org/wiki/Opinion_polling_for_the_1997_United_Kingdom_general_election",     
]


In [209]:
UK_2024_data = scrape_wikipedia_table(urls[0])

<Response [200]>
Web Scraping processing took: 4.7454 seconds


In [211]:
UK_2024_data

,Datesconducted,Pollster,Client,Area,Samplesize,Part1,Part2,Part3,Part4,Part5,Part6
0,,,,,,,None,None,None,None,None
1,4 Jul 2024,2024 general election,–,UK,–,23.7%,33.7%,12.2%,2.5%,6.8%,14.3%
2,GB,24.4%,34.7%,12.5%,2.5%,6.9%,14.7%,4.3%,10.3,None,None
3,3 Jul,Number Cruncher Politics,N/A,GB,"2,496",23%,41%,11%,2%,7%,13%
4,2–3 Jul,JL Partners,The Rest Is Politics,GB,"2,005",23%,38%,13%,3%,5%,17%
...,...,...,...,...,...,...,...,...,...,...,...
324,7 Jan,Redfield & Wilton,N/A,GB,"2,000",27%,43%,10%,3%,5%,11%
325,5–7 Jan,Savanta,N/A,UK,"2,268",26%,45%,10%,3%,5%,8%
326,4–5 Jan,We Think,N/A,GB,"1,226",25%,47%,9%,2%,5%,10%
327,12 Dec–4 Jan,YouGov(MRP)[c],Conservative Britain Alliance[10],GB,"14,110",26%,39.5%,12.5%,3%,7.5%,9%


In [213]:
UK_2024_data = clean_date_column(UK_2024_data, 'UK_2024_data')

In [215]:
UK_2024_data

,Datesconducted,Pollster,Client,Area,Samplesize,Part1,Part2,Part3,Part4,Part5,Part6
0,,,,,,,None,None,None,None,None
1,7/2024,2024 general election,–,UK,–,23.7%,33.7%,12.2%,2.5%,6.8%,14.3%
2,GB,24.4%,34.7%,12.5%,2.5%,6.9%,14.7%,4.3%,10.3,None,None
3,7/2024,Number Cruncher Politics,N/A,GB,"2,496",23%,41%,11%,2%,7%,13%
4,7/2024,JL Partners,The Rest Is Politics,GB,"2,005",23%,38%,13%,3%,5%,17%
...,...,...,...,...,...,...,...,...,...,...,...
324,1/2024,Redfield & Wilton,N/A,GB,"2,000",27%,43%,10%,3%,5%,11%
325,1/2024,Savanta,N/A,UK,"2,268",26%,45%,10%,3%,5%,8%
326,1/2024,We Think,N/A,GB,"1,226",25%,47%,9%,2%,5%,10%
327,12/2024,YouGov(MRP)[c],Conservative Britain Alliance[10],GB,"14,110",26%,39.5%,12.5%,3%,7.5%,9%


In [31]:
# Appeler la fonction pour traiter les URLs
data_frames = process_urls(urls)
variable_name = list(data_frames.keys())

<Response [200]>
Web Scraping processing took: 3.1354 seconds
<Response [200]>
Web Scraping processing took: 1.2809 seconds
<Response [200]>
Web Scraping processing took: 0.7098 seconds
<Response [200]>
Web Scraping processing took: 2.0383 seconds
<Response [200]>
Web Scraping processing took: 0.8149 seconds
<Response [200]>
Web Scraping processing took: 0.2796 seconds
<Response [200]>
Web Scraping processing took: 0.3576 seconds


In [32]:
print(variable_name)

['data_poll_United Kingdom general election_2024', 'data_poll_United Kingdom general election_2019', 'data_poll_United Kingdom general election_2017', 'data_poll_United Kingdom general election_2015', 'data_poll_United Kingdom general election_2010', 'data_poll_United Kingdom general election_2005', 'data_poll_United Kingdom general election_1997']


### Opinion polling for the 2024 United Kingdom general election


In [34]:
data_poll_UK_2024 = data_frames["data_poll_United Kingdom general election_2024"]

In [35]:
data_poll_UK_2024 = cleaner (data_poll_UK_2024, 'data_poll_UK_2024')


C:\Users\sodji\AppData\Local\Temp\ipykernel_2332\3744424348.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace("None", 0, inplace=True)
C:\Users\sodji\AppData\Local\Temp\ipykernel_2332\3744424348.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(0, inplace=True)


In [36]:
# parties
parties2024 = {"Con": "centre-right", "Lab": "Centre-left", "Lib Dems": "Centre", "SNP": "Centre-left (Scottish Nationalist)", "Green": "Ecologist", "Reform": "Right-wing" }
data_poll_UK_2024 = enrichir_dataframe(data_poll_UK_2024, parties2024)
data_poll_UK_2024 = reordonner_colonnes(data_poll_UK_2024)

In [37]:
data_poll_UK_2024 = data_poll_UK_2024.iloc[2:].reset_index(drop=True)
data_poll_UK_2024 = data_poll_UK_2024[data_poll_UK_2024["sample_size"].notna()].reset_index(drop=True)

In [38]:
data_poll_UK_2024 = extract_numbers_from_column(data_poll_UK_2024)

In [39]:
data_poll_UK_2024 = process_column_type_conversion(data_poll_UK_2024)

In [40]:
# Suppression de toutes les lignes contenant des valeurs manquantes
data_poll_UK_2024 = data_poll_UK_2024.dropna()

In [41]:
data_poll_UK_2024.head()

,poll_date,sample_size,polling_organization,prediction_result_candidate_1,final_result_candidate_1,identity_candidate_1,political_learning_candidate_1,prediction_result_candidate_2,final_result_candidate_2,identity_candidate_2,...,identity_candidate_4,political_learning_candidate_4,prediction_result_candidate_5,final_result_candidate_5,identity_candidate_5,political_learning_candidate_5,prediction_result_candidate_6,final_result_candidate_6,identity_candidate_6,political_learning_candidate_6
0,7/2024,2101,Savanta,20,23.7,Con,centre-right,39,33.7,Lab,...,SNP,Centre-left (Scottish Nationalist),5,6.8,Green,Ecologist,17,14.3,Reform,Right-wing
1,6/2024,1022,Survation,18,23.7,Con,centre-right,38,33.7,Lab,...,SNP,Centre-left (Scottish Nationalist),7,6.8,Green,Ecologist,17,14.3,Reform,Right-wing
2,6/2024,2287,Savanta,24,23.7,Con,centre-right,39,33.7,Lab,...,SNP,Centre-left (Scottish Nationalist),4,6.8,Green,Ecologist,13,14.3,Reform,Right-wing
3,6/2024,2092,Savanta,21,23.7,Con,centre-right,38,33.7,Lab,...,SNP,Centre-left (Scottish Nationalist),6,6.8,Green,Ecologist,14,14.3,Reform,Right-wing
4,6/2024,1022,Survation,18,23.7,Con,centre-right,41,33.7,Lab,...,SNP,Centre-left (Scottish Nationalist),5,6.8,Green,Ecologist,14,14.3,Reform,Right-wing


In [42]:
# Save dataset
data_poll_UK_2024.to_excel('UK_2024_general_election.xlsx', index=False)
print(f"Fichier sauvegardé sous : {'UK_2024_general_election.xlsx'}")

Fichier sauvegardé sous : UK_2024_general_election.xlsx


### Opinion polling for the 2019 United Kingdom general election


In [44]:
data_poll_UK_2019 = data_frames["data_poll_United Kingdom general election_2019"]

In [45]:
data_poll_UK_2019 = data_poll_UK_2019.drop(columns=['Part8','Part9'], errors='ignore')
data_poll_UK_2019 = cleaner (data_poll_UK_2019, 'data_poll_UK_2019')

C:\Users\sodji\AppData\Local\Temp\ipykernel_2332\3744424348.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace("None", 0, inplace=True)
C:\Users\sodji\AppData\Local\Temp\ipykernel_2332\3744424348.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(0, inplace=True)


In [46]:
# parties
parties2019 = {"Con": "centre-right", "Lab": "Centre-left", "Lib Dems": "Centre", "SNP": "Centre-left (Scottish Nationalist)", "Plaid Cymru": "centre-left", "Green": "Ecologist", "Reform": "Right-wing" }
data_poll_UK_2019 = enrichir_dataframe(data_poll_UK_2019, parties2019)
data_poll_UK_2019 = reordonner_colonnes(data_poll_UK_2019)

In [47]:
data_poll_UK_2019 = data_poll_UK_2019.iloc[2:].reset_index(drop=True)
data_poll_UK_2019 = data_poll_UK_2019[data_poll_UK_2019["sample_size"].notna()].reset_index(drop=True)

In [48]:
data_poll_UK_2019 = data_poll_UK_2019[~data_poll_UK_2019["sample_size"].astype(str).str.contains("%", na=False)].reset_index(drop=True)
data_poll_UK_2019 = data_poll_UK_2019[~data_poll_UK_2019["poll_date"].astype(str).str.contains("%", na=False)].reset_index(drop=True)

In [49]:
data_poll_UK_2019 = extract_numbers_from_column(data_poll_UK_2019)

In [50]:
data_poll_UK_2019 = process_column_type_conversion(data_poll_UK_2019)

In [51]:
# Suppression de toutes les lignes contenant des valeurs manquantes
data_poll_UK_2019 = data_poll_UK_2019.dropna()

In [52]:
data_poll_UK_2019.head()

,poll_date,sample_size,polling_organization,prediction_result_candidate_1,final_result_candidate_1,identity_candidate_1,political_learning_candidate_1,prediction_result_candidate_2,final_result_candidate_2,identity_candidate_2,...,identity_candidate_5,political_learning_candidate_5,prediction_result_candidate_6,final_result_candidate_6,identity_candidate_6,political_learning_candidate_6,prediction_result_candidate_7,final_result_candidate_7,identity_candidate_7,political_learning_candidate_7
1,12/2019,1012,Survation,45,43.6,Con,centre-right,31,32.1,Lab,...,Plaid Cymru,centre-left,2.0,2.7,Green,Ecologist,4.0,2.0,Reform,Right-wing
2,11/2019,1065,Survation,42,43.6,Con,centre-right,33,32.1,Lab,...,Plaid Cymru,centre-left,4.0,2.7,Green,Ecologist,3.0,2.0,Reform,Right-wing
3,11/2019,1010,Survation,41,43.6,Con,centre-right,30,32.1,Lab,...,Plaid Cymru,centre-left,3.0,2.7,Green,Ecologist,5.0,2.0,Reform,Right-wing
4,11/2019,1010,Survation,42,43.6,Con,centre-right,28,32.1,Lab,...,Plaid Cymru,centre-left,3.0,2.7,Green,Ecologist,5.0,2.0,Reform,Right-wing
5,11/2019,0,0,0,43.6,Con,centre-right,0,32.1,Lab,...,Plaid Cymru,centre-left,0.0,2.7,Green,Ecologist,0.0,2.0,Reform,Right-wing


In [53]:
# Save dataset
data_poll_UK_2019.to_excel('UK_2019_general_election.xlsx', index=False)
print(f"Fichier sauvegardé sous : {'UK_2019_general_election.xlsx'}")

Fichier sauvegardé sous : UK_2019_general_election.xlsx


### Opinion polling for the 2017 United Kingdom general election


In [55]:
data_poll_UK_2017 = data_frames["data_poll_United Kingdom general election_2017"]

In [56]:
data_poll_UK_2017 = cleaner (data_poll_UK_2017, 'data_poll_UK_2017')

C:\Users\sodji\AppData\Local\Temp\ipykernel_2332\3744424348.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace("None", 0, inplace=True)
C:\Users\sodji\AppData\Local\Temp\ipykernel_2332\3744424348.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(0, inplace=True)


In [57]:
# parties
parties2017 = {"Con": "centre-right", "Lab": "Centre-left", "UKIP": "right-wing", "Lib Dems": "Centre", "SNP": "Centre-left (Scottish Nationalist)", "Green": "Ecologist" }
data_poll_UK_2017 = enrichir_dataframe(data_poll_UK_2017, parties2017)
data_poll_UK_2017 = reordonner_colonnes(data_poll_UK_2017)

In [58]:
data_poll_UK_2017 = data_poll_UK_2017.iloc[1:].reset_index(drop=True)
data_poll_UK_2017 = data_poll_UK_2017[data_poll_UK_2017["sample_size"].notna()].reset_index(drop=True)

In [59]:
data_poll_UK_2017 = extract_numbers_from_column(data_poll_UK_2017)

In [60]:
data_poll_UK_2017 = process_column_type_conversion(data_poll_UK_2017)

In [61]:
# Suppression de toutes les lignes contenant des valeurs manquantes
data_poll_UK_2017 = data_poll_UK_2017.dropna()

In [62]:
data_poll_UK_2017.head()

,poll_date,sample_size,polling_organization,prediction_result_candidate_1,final_result_candidate_1,identity_candidate_1,political_learning_candidate_1,prediction_result_candidate_2,final_result_candidate_2,identity_candidate_2,...,identity_candidate_4,political_learning_candidate_4,prediction_result_candidate_5,final_result_candidate_5,identity_candidate_5,political_learning_candidate_5,prediction_result_candidate_6,final_result_candidate_6,identity_candidate_6,political_learning_candidate_6
0,6/2017,1291,Ipsos MORI,44,43.5,Con,centre-right,36,41.0,Lab,...,Lib Dems,Centre,5.0,3.1,SNP,Centre-left (Scottish Nationalist),2,1.7,Green,Ecologist
1,6/2017,1199,BMG,46,43.5,Con,centre-right,33,41.0,Lab,...,Lib Dems,Centre,4.0,3.1,SNP,Centre-left (Scottish Nationalist),3,1.7,Green,Ecologist
2,6/2017,2798,Survation,41,43.5,Con,centre-right,40,41.0,Lab,...,Lib Dems,Centre,4.0,3.1,SNP,Centre-left (Scottish Nationalist),2,1.7,Green,Ecologist
3,6/2017,1532,ICM,46,43.5,Con,centre-right,34,41.0,Lab,...,Lib Dems,Centre,5.0,3.1,SNP,Centre-left (Scottish Nationalist),2,1.7,Green,Ecologist
4,6/2017,2130,YouGov,42,43.5,Con,centre-right,35,41.0,Lab,...,Lib Dems,Centre,5.0,3.1,SNP,Centre-left (Scottish Nationalist),2,1.7,Green,Ecologist


In [63]:
# Save dataset
data_poll_UK_2017.to_excel('UK_2017_general_election.xlsx', index=False)
print(f"Fichier sauvegardé sous : {'UK_2017_general_election.xlsx'}")

Fichier sauvegardé sous : UK_2017_general_election.xlsx


### Opinion polling for the 2015 United Kingdom general election


In [65]:
data_poll_UK_2015 = data_frames["data_poll_United Kingdom general election_2015"]

In [66]:
data_poll_UK_2015 = cleaner (data_poll_UK_2015, 'data_poll_UK_2015')

C:\Users\sodji\AppData\Local\Temp\ipykernel_2332\3744424348.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace("None", 0, inplace=True)
C:\Users\sodji\AppData\Local\Temp\ipykernel_2332\3744424348.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(0, inplace=True)


In [67]:
# parties
parties2015 = {"Con": "centre-right", "Lab": "Centre-left", "Lib Dem": "Centre", "UKIP": "right-wing", "Green": "Ecologist" }
data_poll_UK_2015 = enrichir_dataframe(data_poll_UK_2015, parties2015)
data_poll_UK_2015 = reordonner_colonnes(data_poll_UK_2015)

In [68]:
data_poll_UK_2015 = data_poll_UK_2015.iloc[1:].reset_index(drop=True)
data_poll_UK_2015 = data_poll_UK_2015[data_poll_UK_2015["sample_size"].notna()].reset_index(drop=True)

In [69]:
data_poll_UK_2015 = extract_numbers_from_column(data_poll_UK_2015)

In [70]:
data_poll_UK_2015 = process_column_type_conversion(data_poll_UK_2015)

In [71]:
# Suppression de toutes les lignes contenant des valeurs manquantes
data_poll_UK_2015 = data_poll_UK_2015.dropna()

In [72]:
data_poll_UK_2015.head()

,poll_date,sample_size,polling_organization,prediction_result_candidate_1,final_result_candidate_1,identity_candidate_1,political_learning_candidate_1,prediction_result_candidate_2,final_result_candidate_2,identity_candidate_2,...,identity_candidate_3,political_learning_candidate_3,prediction_result_candidate_4,final_result_candidate_4,identity_candidate_4,political_learning_candidate_4,prediction_result_candidate_5,final_result_candidate_5,identity_candidate_5,political_learning_candidate_5
0,5/2015,3917,Populus,34,37.8,Con,centre-right,34,31.2,Lab,...,Lib Dem,Centre,13,12.9,UKIP,right-wing,5,3.8,Green,Ecologist
1,5/2015,1045,Survation,37,37.8,Con,centre-right,31,31.2,Lab,...,Lib Dem,Centre,11,12.9,UKIP,right-wing,5,3.8,Green,Ecologist
2,4/2015,18131,SurveyMonkey,34,37.8,Con,centre-right,28,31.2,Lab,...,Lib Dem,Centre,13,12.9,UKIP,right-wing,8,3.8,Green,Ecologist
3,5/2015,3028,Lord Ashcroft,33,37.8,Con,centre-right,33,31.2,Lab,...,Lib Dem,Centre,11,12.9,UKIP,right-wing,6,3.8,Green,Ecologist
4,5/2015,1186,Ipsos MORI,36,37.8,Con,centre-right,35,31.2,Lab,...,Lib Dem,Centre,11,12.9,UKIP,right-wing,5,3.8,Green,Ecologist


In [73]:
# Save dataset
data_poll_UK_2015.to_excel('UK_2015_general_election.xlsx', index=False)
print(f"Fichier sauvegardé sous : {'UK_2015_general_election.xlsx'}")

Fichier sauvegardé sous : UK_2015_general_election.xlsx


### Opinion polling for the 2010 United Kingdom general election


In [75]:
data_poll_UK_2010 = data_frames["data_poll_United Kingdom general election_2010"]

In [76]:
data_poll_UK_2010 = cleaner (data_poll_UK_2010, 'data_poll_UK_2010')

In [77]:
# parties
parties2010 = {"Lab": "Centre-left", "Con": "centre-right", "Lib Dem": "Centre"}
data_poll_UK_2010 = enrichir_dataframe(data_poll_UK_2010, parties2010)
data_poll_UK_2010 = reordonner_colonnes(data_poll_UK_2010)

In [78]:
data_poll_UK_2010 = data_poll_UK_2010.iloc[2:].reset_index(drop=True)
data_poll_UK_2010 = data_poll_UK_2010[data_poll_UK_2010["sample_size"].notna()].reset_index(drop=True)

In [79]:
data_poll_UK_2010 = data_poll_UK_2010[~data_poll_UK_2010["poll_date"].astype(str).str.contains("%", na=False)].reset_index(drop=True)

In [80]:
data_poll_UK_2010 = extract_numbers_from_column(data_poll_UK_2010)

In [81]:
data_poll_UK_2010 = process_column_type_conversion(data_poll_UK_2010)

In [82]:
# Suppression de toutes les lignes contenant des valeurs manquantes
data_poll_UK_2010 = data_poll_UK_2010.dropna()

In [83]:
data_poll_UK_2010.head()

,poll_date,sample_size,polling_organization,prediction_result_candidate_1,final_result_candidate_1,identity_candidate_1,political_learning_candidate_1,prediction_result_candidate_2,final_result_candidate_2,identity_candidate_2,political_learning_candidate_2,prediction_result_candidate_3,final_result_candidate_3,identity_candidate_3,political_learning_candidate_3
0,5/2010,1216.0,Ipsos MORI,29,36.9,Lab,Centre-left,36,23.6,Con,centre-right,27,9.8,Lib Dem,Centre
1,5/2010,6483.0,YouGov,28,36.9,Lab,Centre-left,35,23.6,Con,centre-right,28,9.8,Lib Dem,Centre
2,5/2010,4014.0,Harris Interactive,29,36.9,Lab,Centre-left,35,23.6,Con,centre-right,27,9.8,Lib Dem,Centre
3,5/2010,2505.0,PopulusArchived25 July 2018 at theWayback Machine,28,36.9,Lab,Centre-left,37,23.6,Con,centre-right,27,9.8,Lib Dem,Centre
4,5/2010,2283.0,Angus Reid Public Opinion,24,36.9,Lab,Centre-left,36,23.6,Con,centre-right,29,9.8,Lib Dem,Centre


In [84]:
# Save dataset
data_poll_UK_2010.to_excel('UK_2010_general_election.xlsx', index=False)
print(f"Fichier sauvegardé sous : {'UK_2010_general_election.xlsx'}")

Fichier sauvegardé sous : UK_2010_general_election.xlsx


### Opinion polling for the 2005 United Kingdom general election


In [86]:
data_poll_UK_2005 = data_frames["data_poll_United Kingdom general election_2005"]

In [87]:
data_poll_UK_2005 = cleaner (data_poll_UK_2005, 'data_poll_UK_2005')

In [88]:
# parties
parties2005 = {"Lab": "Centre-left", "Con": "centre-right", "Lib Dem": "Centre"}
data_poll_UK_2005 = enrichir_dataframe(data_poll_UK_2005, parties2005)
data_poll_UK_2005 = reordonner_colonnes(data_poll_UK_2005)

In [89]:
data_poll_UK_2005 = data_poll_UK_2005.iloc[2:].reset_index(drop=True)
data_poll_UK_2005 = data_poll_UK_2005[data_poll_UK_2005["sample_size"].notna()].reset_index(drop=True)

In [90]:
data_poll_UK_2005 = extract_numbers_from_column(data_poll_UK_2005)

In [91]:
data_poll_UK_2005 = process_column_type_conversion(data_poll_UK_2005)

In [92]:
# Suppression de toutes les lignes contenant des valeurs manquantes
data_poll_UK_2005 = data_poll_UK_2005.dropna()

In [93]:
data_poll_UK_2005.head()

,poll_date,sample_size,polling_organization,prediction_result_candidate_1,final_result_candidate_1,identity_candidate_1,political_learning_candidate_1,prediction_result_candidate_2,final_result_candidate_2,identity_candidate_2,political_learning_candidate_2,prediction_result_candidate_3,final_result_candidate_3,identity_candidate_3,political_learning_candidate_3
0,5/2005,1164.0,Ipsos MORI,38,33.2,Lab,Centre-left,33,22.7,Con,centre-right,23,7.9,Lib Dem,Centre
1,5/2005,1174.0,Populus,38,33.2,Lab,Centre-left,32,22.7,Con,centre-right,21,7.9,Lib Dem,Centre
2,5/2005,1178.0,ICM,38,33.2,Lab,Centre-left,32,22.7,Con,centre-right,22,7.9,Lib Dem,Centre
3,4/2005,866.0,Populus,41,33.2,Lab,Centre-left,27,22.7,Con,centre-right,23,7.9,Lib Dem,Centre
4,4/2005,863.0,Populus,42,33.2,Lab,Centre-left,29,22.7,Con,centre-right,21,7.9,Lib Dem,Centre


In [94]:
# Save dataset
data_poll_UK_2005.to_excel('UK_2005_general_election.xlsx', index=False)
print(f"Fichier sauvegardé sous : {'UK_2005_general_election.xlsx'}")

Fichier sauvegardé sous : UK_2005_general_election.xlsx


### Opinion polling for the 1997 United Kingdom general election


In [96]:
data_poll_UK_1997 = data_frames["data_poll_United Kingdom general election_1997"]

In [97]:
nouveaux_noms = {
        "Part1": "Datesconducted",
        "Part2": "Pollster",
        "Part3": "Part1",
        "Part4": "Part2",
        "Part5": "Part3",
        "Part6": "Part4"
    }    
data_poll_UK_1997 = data_poll_UK_1997.rename(columns=nouveaux_noms)

In [98]:
 data_poll_UK_1997 = data_poll_UK_1997.drop(columns=["Part4"], errors="ignore")

In [99]:
# Insérer la colonne "sample_size" remplie de 0 à la position après "polling_organization"
data_poll_UK_1997.columns = data_poll_UK_1997.columns.str.strip().str.replace(" ", "_")
col_position = data_poll_UK_1997.columns.get_loc("Pollster") + 1
data_poll_UK_1997.insert(col_position, "Sample", 0)

In [100]:
data_poll_UK_1997 = cleaner (data_poll_UK_1997, 'data_poll_UK_1997')

In [101]:
# parties
parties1997 = {"Con": "centre-right","Lab": "Centre-left", "Lib Dem": "Centre"}
data_poll_UK_1997 = enrichir_dataframe(data_poll_UK_1997, parties1997)
data_poll_UK_1997 = reordonner_colonnes(data_poll_UK_1997)

In [102]:
data_poll_UK_1997 = data_poll_UK_1997.iloc[2:].reset_index(drop=True)
data_poll_UK_1997 = data_poll_UK_1997[data_poll_UK_1997["sample_size"].notna()].reset_index(drop=True)

In [103]:
 data_poll_UK_1997 = data_poll_UK_1997.drop(columns=["sample_size"], errors="ignore")

In [104]:
data_poll_UK_1997 = extract_numbers_from_column(data_poll_UK_1997)

In [105]:
data_poll_UK_1997 = process_column_type_conversion(data_poll_UK_1997)

In [106]:
# Suppression de toutes les lignes contenant des valeurs manquantes
data_poll_UK_1997 = data_poll_UK_1997.dropna()

In [107]:
data_poll_UK_1997.head()

,poll_date,polling_organization,prediction_result_candidate_1,final_result_candidate_1,identity_candidate_1,political_learning_candidate_1,prediction_result_candidate_2,final_result_candidate_2,identity_candidate_2,political_learning_candidate_2,prediction_result_candidate_3,final_result_candidate_3,identity_candidate_3,political_learning_candidate_3
0,4/1997,ICM,33,30.7,Con,centre-right,43,43.2,Lab,Centre-left,18,16.8,Lib Dem,Centre
1,4/1997,Gallup,33,30.7,Con,centre-right,46,43.2,Lab,Centre-left,16,16.8,Lib Dem,Centre
2,4/1997,MORI,29,30.7,Con,centre-right,47,43.2,Lab,Centre-left,19,16.8,Lib Dem,Centre
3,4/1997,Gallup,31,30.7,Con,centre-right,51,43.2,Lab,Centre-left,13,16.8,Lib Dem,Centre
4,4/1997,Harris,31,30.7,Con,centre-right,48,43.2,Lab,Centre-left,15,16.8,Lib Dem,Centre


In [108]:
# Save dataset
data_poll_UK_1997.to_excel('UK_1997_general_election.xlsx', index=False)
print(f"Fichier sauvegardé sous : {'UK_1997_general_election.xlsx'}")

Fichier sauvegardé sous : UK_1997_general_election.xlsx
